**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 8 - Open Science Platform - Provenance

In [7]:
import json
import datetime
from loguru import logger
from integration_helpers import get_engine_receipts_result

def process_engine_receipts(input_json_file, output_json_file):
    try:
        # Load the input JSON file
        with open(input_json_file, "r") as f:
            account_data = json.load(f)
        
        result_data = {}
        
        # Iterate through each account and its transaction logs
        for account, transactions in account_data.items():
            result_data[account] = []
            
            for entry in transactions:
                tx_hash = entry["hex_hash"]
                timestamp = entry["timestamp"]
                
                # Convert timestamp to UTC using timezone-aware datetime
                utc_time = datetime.datetime.fromtimestamp(timestamp / 1000, datetime.timezone.utc).isoformat()

                
                # Get decoded result from transaction hash
                decoded_string = get_engine_receipts_result(tx_hash)
                
                # Append new data to the result structure
                result_data[account].append({
                    "address": entry["address"],
                    "hex_hash": tx_hash,
                    "timestamp": timestamp,
                    "timestamp_utc": utc_time,
                    "decoded_string": decoded_string
                })
        
        # Write the output JSON file
        with open(output_json_file, "w") as f:
            json.dump(result_data, f, indent=4)
        
        logger.info(f"Decoded results saved to {output_json_file}")
    except Exception as e:
        logger.error(f"Error processing engine receipts: {e}")


In [8]:
input_json_file="logs/account_data.json"

output_json_file="logs/decoded_results.json"

process_engine_receipts(input_json_file, output_json_file)

2025-02-09 10:07:55.393 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-09 10:07:55.413 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000547b226163636f756e745f6964223a2262656175746966756c5f656c6c69734074657374222c22646f6d61696e5f6964223a2274657374222c2271756f72756d223a312c226a736f6e5f64617461223a227b7d227d000000000000000000000000
2025-02-09 10:07:55.414 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                                               T{"account_id":"beautiful_ellis@test","domain_id":"test","quorum":1,"json_data":"{}"}            
202

2025-02-09 10:07:55.532 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 00000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000a87b226163636f756e745f6964223a2232303134364074657374222c22646f6d61696e5f6964223a2274657374222c2271756f72756d223a312c226a736f6e5f64617461223a227b5c2261646d696e40746573745c223a207b5c226163636f756e745f6d657461646174615f6369645c223a205c22516d5344533155665944727675716568645350737a6b466e5871486b41784464536f47336538703855586d67366b5c227d7d227d000000000000000000000000000000000000000000000000
2025-02-09 10:07:55.533 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                                               {"account_id":"2